In [12]:
using BlockBandedMatrices, LinearAlgebra, Test

Suppose the symbols $x(z) = B^x/z + A^x + \left(B^x\right)^{\intercal}z$ and $y(z) = B^y/z + A^y + \left(B^y\right)^{\intercal}z$ commute, then

$$
B^xB^y = B^yB^x
$$

$$
A^xB^y + B^x A^y  = A^yB^x + B^y A^x
$$

$$
\left( B^x \right)^{\intercal}B^y + A^xA^y + B^x \left( B^y \right)^{\intercal} = 
\left( B^y \right)^{\intercal}B^x + A^yA^x + B^y \left( B^x \right)^{\intercal}
$$

Define the truncated $X$ and $Y$ Toeplitz operators:

$$
\tilde{X} = \left(
\begin{array}{c c c  c}
A^x  & B^x   &        &           \\
\left(B^x \right)^{\intercal} & \ddots  & \ddots &              \\
                                &\ddots & \ddots  &                   B^x  \\
                                &                                  & \left(B^x \right)^{\intercal} & A^x
\end{array}
\right), \qquad
\tilde{Y} = \left(
\begin{array}{c c c  c}
A^y  & B^y   &        &           \\
\left(B^y \right)^{\intercal} & \ddots  & \ddots &              \\
                                &\ddots & \ddots  &                   B^y  \\
                                &                                  & \left(B^y \right)^{\intercal} & A^y
\end{array}
\right),
$$

The commutator of $\tilde{X}$ and $\tilde{Y}$ has only two nonzero blocks (top-left and bottom-right corners): 

$$
\left[ \tilde{X}, \tilde{Y} \right] = \tilde{X} \tilde{Y} -  \tilde{Y} \tilde{X} = 
\left(
\begin{array}{c c c c c}
\left(B^y\right)^{\intercal}B^x - \left(B^x\right)^{\intercal}B^y & & & & \\
& O & & & \\
& & \ddots & & \\
& &        & O & \\
& & & & B^y\left(B^x\right)^{\intercal} - B^x\left(B^y\right)^{\intercal}
\end{array}
\right).
$$

Note that if $B^x$ and $B^y$ are symmetric, then $\left[ \tilde{X}, \tilde{Y} \right] = O$. If $\left(B^x\right)^{\intercal}B^y$ and $B^x\left(B^y\right)^{\intercal}$ are symmetric, then $\left[ \tilde{X}, \tilde{Y} \right] = O$. 

Take as an example the half-range Chebyshev OPs on the semicircle:

In [13]:
Aˣ = Matrix(0.5I,2,2)
Bˣ = Matrix(0.25I,2,2)
a₁₂ = (1 + sqrt(2))/4
a₂₁ = (1 - sqrt(2))/4
Aʸ = [0 -0.5; -0.5 0]
Bʸ = [0 a₁₂; a₂₁ 0]
bs = 4
Xt = BlockBandedMatrix(zeros(bs*2,bs*2),fill(2,bs),fill(2,bs),(1,1))
for b = 1:bs-1
    Xt[Block(b,b)]= Aˣ
    Xt[Block(b,b+1)]= Bˣ; Xt[Block(b+1,b)]= Bˣ' 
end
Xt[Block(bs,bs)]=Aˣ
Yt = BlockBandedMatrix(zeros(bs*2,bs*2),fill(2,bs),fill(2,bs),(1,1))
for b = 1:bs-1
    Yt[Block(b,b)]= Aʸ
    Yt[Block(b,b+1)]= Bʸ; Yt[Block(b+1,b)]= Bʸ' 
end
Yt[Block(bs,bs)]=Aʸ
commutator = Xt*Yt-Yt*Xt
commutator_exact = BlockBandedMatrix(zeros(bs*2,bs*2),fill(2,bs),fill(2,bs),(1,1))
commutator_exact[Block(1,1)]   = Bʸ'*Bˣ - Bˣ'*Bʸ
commutator_exact[Block(bs,bs)] = Bʸ*Bˣ' - Bˣ*Bʸ'
@test commutator ≈ commutator_exact

Test Passed

Is it possible to perturb  $\tilde{X}$ and $\tilde{Y}$ such that they commute? 

# $2\times 2$ case

Define the truncated and perturbed operators

$$
\hat{X} := \left(
\begin{array}{c c c  c c c}
A_0^x  & B_1^x   &        &        & &    \\
\left(B_1^x \right)^{\intercal} & A^x  & B^x &       & &        \\
                                &\left(B^x \right)^{\intercal} & \ddots  &  \ddots        &  &         \\
                                            & & \ddots  &  \ddots        & B^x  &          \\
             & &   &    \left(B^x \right)^{\intercal}      & A^x &         \left(b_1^x \right)^{\intercal}  \\
                                &                        & &          & b_1^x & a_0^x
\end{array}
\right), \qquad
\hat{Y} := \left(
\begin{array}{c c c  c c c}
A_0^y  & B_1^y   &        &        & &    \\
\left(B_1^y \right)^{\intercal} & A^y  & B^y &       & &        \\
                                &\left(B^y \right)^{\intercal} & \ddots  &  \ddots        &  &         \\
                                            & & \ddots  &  \ddots        & B^y  &          \\
             & &   &    \left(B^y \right)^{\intercal}      & A^y &         \left(b_1^y \right)^{\intercal}  \\
                                &                        & &          & b_1^y & a_0^y
\end{array}
\right),
$$

where $A_0^x, a_0^x, A_0^y,  a_0^y$ are $1\times 1$ and $B_1^x, b_1^x, B_1^y, b_1^y$ are $1 \times 2$ matrices whose entries remain to be determined; $A^x, B^x, A^y, B^y$ are given $2 \times 2$ matrices that satisfy the above equations. The commutator of $\hat{X}$ and $\hat{Y}$ is
$$
\left[\hat{X}, \hat{Y}  \right]= \hat{X}\hat{Y} - \hat{Y}\hat{X}  =
\left(
\begin{array}{c c c c c c}
 0 & C_1 & C_2 &  & & \\
C_1^{\intercal} & C_3 &   &   & & \\
C_2^{\intercal} &     &    &   &  &   \\
  &   &    &   &  & D_2   \\
  &   &    &   & D_3 & D_1 \\
  &   &    & D_2^{\intercal} & D_1^{\intercal} & 0
\end{array}
\right),
$$
where

\begin{eqnarray}
C_1 &=&  B_1^xA^y + A_0^x B_1^y -  \left[ B_1^yA^x + A_0^y B_1^x \right] \\
C_2 &=& B_1^x B^y -  B_1^y B^x \\
C_3 &=& A^x A^y + B^x\left( B^y\right)^{\intercal} + \left( B_1^x\right)^{\intercal}B_1^y  - \left[A^y A^x + B^y\left( B^x\right)^{\intercal} + \left( B_1^y\right)^{\intercal}B_1^x  \right] \\
&=& \left(B^y\right)^{\intercal}B^x + \left( B_1^x\right)^{\intercal}B_1^y - \left[ \left(B^x\right)^{\intercal}B^y +  \left( B_1^y\right)^{\intercal}B_1^x \right]\\
D_1^{\intercal} &=&  b_1^xA^y + a_0^x b_1^y -  \left[ b_1^yA^x + a_0^y b_1^x \right]\\
D_2^{\intercal} &=& b_1^x \left(B^y\right)^{\intercal} - b_1^y \left(B^x\right)^{\intercal} \\
D_3 &=& A^x A^y + \left( B^x\right)^{\intercal}B^y + \left( b_1^x\right)^{\intercal}b_1^y  - \left[A^y A^x + \left( B^y\right)^{\intercal}B^x + \left( b_1^y\right)^{\intercal}b_1^x  \right] \\
&=& B^y\left(B^x\right)^{\intercal} + \left( b_1^x\right)^{\intercal}b_1^y - \left[ B^x\left(B^y\right)^{\intercal} +  \left( b_1^y\right)^{\intercal}b_1^x \right]
\end{eqnarray}

In [14]:
@testset "2x2 truncated & perturbed commutator" begin
x₂ = -sqrt(4+2sqrt(2))/4
x₀ = -1/sqrt(2)
A0x = [x₀]; a0x = [x₀]
B1x = [0 x₂]; b1x = [x₂ 0]
A0y = [0]; a0y = [0]
y₁=sqrt(4-2sqrt(2))/4
B1y = [y₁ 0]; b1y = [0 y₁]
bs = 4
X = BlockBandedMatrix(zeros(2(bs+1),2(bs+1)),[1;fill(2,bs);1],[1;fill(2,bs);1],(1,1))
Y = BlockBandedMatrix(zeros(2(bs+1),2(bs+1)),[1;fill(2,bs);1],[1;fill(2,bs);1],(1,1))
X[Block(1,1)]=A0x; X[Block(bs+2,bs+2)] = a0x
X[Block(1,2)]=B1x; X[Block(2,1)] = B1x'
X[Block(bs+2,bs+1)]=b1x; X[Block(bs+1,bs+2)]=b1x'
Y[Block(1,1)]=A0y; Y[Block(bs+2,bs+2)] = a0y
Y[Block(1,2)]=B1y; Y[Block(2,1)] = B1y'
Y[Block(bs+2,bs+1)]=b1y; Y[Block(bs+1,bs+2)]=b1y'
for b = 1:bs-1
    X[Block(b+1,b+1)]= Aˣ
    X[Block(b+1,b+2)]= Bˣ; X[Block(b+2,b+1)]= Bˣ' 
end
X[Block(bs+1,bs+1)]=Aˣ
for b = 1:bs-1
    Y[Block(b+1,b+1)]=Aʸ
    Y[Block(b+1,b+2)]= Bʸ; Y[Block(b+2,b+1)]= Bʸ' 
end
Y[Block(bs+1,bs+1)]=Aʸ
@test X*Y ≈ X*Y
@test B1x*Ay + A0x*B1y ≈ B1y*Ax + A0y*B1x 
@test B1x*By ≈ B1y*Bx 
@test By'*Bx + B1x'*B1y ≈ Bx'*By + B1y'*B1x    
@test b1x*Ay + a0x*b1y ≈ b1y*Ax + a0y*b1x 
@test b1x*By' ≈ b1y*Bx' 
@test By*Bx' + b1x'*b1y ≈ Bx*By' + b1y'*b1x
end;

Test Summary:                        | Pass  Total
2x2 truncated & perturbed commutator |    7      7


# $3 \times 3$ case

Define the truncated and perturbed operators

$$
\hat{X} := \left(
\begin{array}{c c c c c c c c }
A_0^x  & B_1^x   &        &        & & & &   \\
\left(B_1^x \right)^{\intercal} & A_1^x  & B_2^x &       & &  & &      \\
                                &\left(B_2^x \right)^{\intercal} & A^x  &  B^x       &  &    & &     \\
                                            & & \left(B^x \right)^{\intercal}  &  \ddots        & \ddots  &  & &       \\
                                                        & &    &   \ddots      & \ddots  &B^x   & &       \\
             & &   &          & \left(B^x \right)^{\intercal}  &      A^x    & b_2^x &  \\
                                &                        & &          &  & \left(b_2^x \right)^{\intercal}  & a_1^x & b_1^x  \\
                                &     & & & &  & \left(b_1^x \right)^{\intercal}  & a_0^x
\end{array}
\right), 
\qquad
\hat{Y} := \left(
\begin{array}{c c c c c c c c }
A_0^y  & B_1^y   &        &        & & & &   \\
\left(B_1^y \right)^{\intercal} & A_1^y  & B_2^y &       & &  & &      \\
                                &\left(B_2^y \right)^{\intercal} & A^y  &  B^y       &  &    & &     \\
                                            & & \left(B^y \right)^{\intercal}  &  \ddots        & \ddots  &  & &       \\
                                                        & &    &   \ddots      & \ddots  &B^y   & &       \\
             & &   &          & \left(B^y \right)^{\intercal}  &      A^y    & b_2^y &  \\
                                &                        & &          &  & \left(b_2^y \right)^{\intercal}  & a_1^y & b_1^y  \\
                                &     & & & &  & \left(b_1^y \right)^{\intercal}  & a_0^y
\end{array}
\right),
$$

where $A_0^x, a_0^x, A_0^y,  a_0^y$ are $1\times 1$; $B_1^x, b_1^x, B_1^y, b_1^y$ are $1 \times 2$ and $B_2^x, b_2^x, B_2^y, b_2^y$ are $2 \times 3$ matrices whose entries remain to be determined; $A^x, B^x, A^y, B^y$ are given $3 \times 3$ matrices that satisfy the above equations. The block-pentadiagonal commutator of $\hat{X}$ and $\hat{Y}$ is

$$
\left[\hat{X}, \hat{Y}  \right]= \hat{X}\hat{Y} - \hat{Y}\hat{X}  =
\left(
\begin{array}{c c c c c c c c}
 0 & C_1 & C_2 &  & & & & \\
C_1^{\intercal} & C_3 & C_5  & C_4   & & & & \\
C_2^{\intercal} & C_5^{\intercal} & C_6 &   &  & & &   \\
 & C_4^{\intercal} & & & & & &   \\
 & & & & & &D_4 &  \\
      &   &  & &  & D_6  & D_5 & D_2    \\
  &   &  & & D_4^{\intercal} & D_5^{\intercal}  & D_3 & D_1 \\
  &   &  & &  & D_2^{\intercal} & D_1^{\intercal} & 0
\end{array}
\right),
$$
where

\begin{eqnarray}
C_1 &=&  B_1^xA_1^y + A_0^x B_1^y -  \left[ B_1^yA_1^x + A_0^y B_1^x \right]\\
C_2 &=& B_1^x B_2^y -  B_1^y B_2^x \\
C_3 &=& A_1^x A_1^y + B_2^x \left( B_2^y\right)^{\intercal}+ \left( B_1^x\right)^{\intercal}B_1^y  - \left[A_1^y A_1^x + B_2^y\left( B_2^x\right)^{\intercal} + \left( B_1^y\right)^{\intercal}B_1^x  \right] \\
C_4 &=& B_2^x A^y + A_1^x B_2^y - \left[ B_2^y A^x + A_1^y B_2^x \right] \\
C_5 &=& B_2^x B^y - B_2^y B^x \\
C_6 &=& A^x A^y + B^x\left( B^y\right)^{\intercal} + \left( B_2^x\right)^{\intercal}B_2^y  - \left[A^y A^x + B^y\left( B^x\right)^{\intercal} + \left( B_2^y\right)^{\intercal}B_2^x  \right] \\
&=& \left(B^y\right)^{\intercal}B^x + \left( B_2^x\right)^{\intercal}B_2^y - \left[ \left(B^x\right)^{\intercal}B^y + \left( B_2^y\right)^{\intercal}B_2^x \right] \\
D_1^{\intercal} &=&  b_1^xa_1^y + a_0^x b_1^y -  \left[ b_1^ya_1^x + a_0^y b_1^x \right]\\
D_2^{\intercal} &=& b_1^x b_2^y - b_1^y b_2^x \\
D_3 &=& a_1^x a_1^y + b_2^x \left( b_2^y\right)^{\intercal}+ \left( b_1^x\right)^{\intercal}b_1^y  - \left[a_1^y a_1^x + b_2^y\left( b_2^x\right)^{\intercal} + \left( b_1^y\right)^{\intercal}b_1^x  \right] \\
D_4^{\intercal} &=& b_2^x A^y + a_1^x b_2^y - \left[ b_2^y A^x + a_1^y b_2^x \right] \\
D_5^{\intercal} &=& b_2^x \left(B^y\right)^{\intercal} - b_2^y \left(B^x\right)^{\intercal} \\
D_6 &=& A^x A^y + \left( B^x\right)^{\intercal}B^y + \left( b_2^x\right)^{\intercal}b_2^y  - \left[A^y A^x + \left( B^y\right)^{\intercal}B^x + \left( b_2^y\right)^{\intercal}b_2^x  \right] \\
&=& B^y\left(B^x\right)^{\intercal} + \left( b_2^x\right)^{\intercal}b_2^y - \left[ B^x\left(B^y\right)^{\intercal} + \left( b_2^y\right)^{\intercal}b_2^x \right]
\end{eqnarray}